# GEOPARSING HISTORICAL DOCUMENTS

This notebook is proposed by [L. Moncla](https://ludovicmoncla.github.io/) and [K. McDonough](https://www.turing.ac.uk/people/researchers/katherine-mcdonough) as part of the [Sunoikisis Digital Classics](https://github.com/SunoikisisDC/SunoikisisDC-2021-2022/wiki/SunoikisisDC-Summer-2022-Session-9) Summer course on NLP for Historical maps (Session 9).

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](http://colab.research.google.com/github/ludovicmoncla/SunoikisisDC-Summer2022-Session9/blob/main/GeoparsingEncyclopedie.ipynb)
[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/ludovicmoncla/SunoikisisDC-Summer2022-Session9/master?filepath=GeoparsingEncyclopedie.ipynb)


## Overview

In this tutorial, we'll learn about a few different things.

- How to load data from TEI-XML files into a Python dataframe
- Use Python dataframe for simple data analysis
- Test the [Perdido](https://github.com/ludovicmoncla/perdido) for geoparsing (geotagging + geocoding) Encyclopedie articles
- Display custom geotagging results (PERDIDO TEI-XML) with the [displaCy Named Entity Visualizer](https://spacy.io/usage/visualizers)
- Display geocoding results on a map
- Discuss the limits of geoparsing historical French texts

## Introduction

Geoparsing (also known as toponym resolution) refers to the process of extracting place names from text and assigning geographic coordinates to them.
This involves two main tasks: geotagging and geocoding.
Geotagging consists to identify spans of text referring to place names while geocoding consists to find unambiguous geographic coordinates.


### The Perdido Geoparser python library

[Perdido](https://github.com/ludovicmoncla/perdido) is a python text geoparser. It provides NLP and GIS methods for geoparsing French texts.
It has initially been developed as a REST API for extracting and retrieving displacements from French hiking descriptions [PERDIDO](http://erig.univ-pau.fr/PERDIDO/) and [ANR Choucas](http://choucas.ign.fr) projects.

More recently, as part of the [GEODE project](https://geode-project.github.io) we have developed a custom version for historical documents and more specifically for the Encyclopédie.


In this tutorial we'll see how to use the `Perdido` python library for geoparsing French texts. 
We will apply geoparsing on the Encyclopedie corpus version released by the [ARTFL project](https://encyclopedie.uchicago.edu/) and we'll show the limit of geotagging and geocoding historical documents.

### Acknowledgement

Data courtesy the [ARTFL Encyclopédie Project](https://artfl-project.uchicago.edu/), University of Chicago.


## Setting up the environment



### Install the Perdido Geoparser python library

In [ ]:
!pip install --upgrade perdido

### Import the libraries

In [1]:
from perdido.geoparser import Geoparser
from perdido.datasets import load_edda_artfl, load_edda_perdido

import lxml.etree as etree
from spacy import displacy


## 1. Getting started

### 1.1 Loading the Encyclopédie ARTFL dataset



In [2]:
d = load_edda_artfl()
dataset = d['data']
dataset.head()

,filename,volume,number,head,normClass,author,text
0,volume01-1063.tei,1,1063,AFRIQUE,Géographie,Diderot,"* AFRIQUE, (Géog.) l'une des quatre parties pr..."
1,volume01-1468.tei,1,1468,AISNE,Géographie,Diderot,"* AISNE, (Géog.) riviere de France, qui a sa s..."
2,volume01-1789.tei,1,1789,ALLEMAGNE,Géographie,Diderot,"* ALLEMAGNE, (Geog.) grand pays situé au milie..."
3,volume01-2568.tei,1,2568,ANCUD,Géographie moderne,Diderot,"* ANCUD, (Géog. mod.) l'Archipel d'Ancud ou de..."
4,volume01-4093.tei,1,4093,ARRACIFES,Géographie,Diderot,"* ARRACIFES, (Géog.) une des îles des Larrons,..."


### 1.2 First look at the data

Now we have access to all the attributs and methods of the dataframe object. For instance, we can easily print the number of rows in our dataframe which correspond to the number of articles in our corpus:

In [3]:
n = dataset.shape[0]
print('There are ' + str(n) + ' articles in the input directory')

There are 35 articles in the input directory




Now that the data from the XML-TEI files are loaded into a python dataframe, we can have a look at them.
For instance, we can select articles based on their classification in the Encyclopedie.
If we want all articles in 'geography' we can just do as follows: 

In [4]:
# create the list of class that refers to 'Géographie'
normclassGEO = ['Géographie', 'Géographie moderne',
                 'Géographie ancienne', 'Géographie moderne | Géographie ancienne',
                 'Géographie ancienne | Géographie moderne', 'Géographie sacrée', 'Géographie sainte',
                 'Géographie | Histoire ancienne', 'Géographie historique', 'Géographie | Histoire',
                 'Histoire | Géographie', 'Géographie | Histoire naturelle', 'Géographie | Mythologie',
                 'Géographie ancienne | Mythologie', 'Histoire moderne | Géographie',
                 'Géographie ancienne | Géographie sainte', 'Géographie ancienne | Géographie sacrée',
                 'Géographie sacrée | Géographie ancienne', 'Géographie du moyen âge', 'Géographie des Arabes',
                 'Géographie | Commerce', 'Histoire | Géographie ancienne',
                 'Géographie | Histoire ancienne | Histoire moderne', 'Géographie ancienne | Littérature | Histoire',
                 'Histoire naturelle | Géographie', 'Géographie | Histoire ancienne | Mythologie',
                 'Géographie moderne | Commerce', 'Géographie ancienne | Géographie antique',
                 'Géographie moderne | Histoire', 'Géographie | Histoire monastique',
                 'Géographie ancienne | Géographie moderne | Mythologie', 'Géographie ancienne | Histoire',
                 'Géographie ancienne | Littérature | Mythologie', 'Géographie ancienne | Médailles'
                 ]

# query the dataframe for all articles matching one of the class in our list
df_geo = dataset.loc[dataset['normClass'].isin(normclassGEO)]
df_geo.head(10)

,filename,volume,number,head,normClass,author,text
0,volume01-1063.tei,1,1063,AFRIQUE,Géographie,Diderot,"* AFRIQUE, (Géog.) l'une des quatre parties pr..."
1,volume01-1468.tei,1,1468,AISNE,Géographie,Diderot,"* AISNE, (Géog.) riviere de France, qui a sa s..."
2,volume01-1789.tei,1,1789,ALLEMAGNE,Géographie,Diderot,"* ALLEMAGNE, (Geog.) grand pays situé au milie..."
3,volume01-2568.tei,1,2568,ANCUD,Géographie moderne,Diderot,"* ANCUD, (Géog. mod.) l'Archipel d'Ancud ou de..."
4,volume01-4093.tei,1,4093,ARRACIFES,Géographie,Diderot,"* ARRACIFES, (Géog.) une des îles des Larrons,..."
5,volume01-4583.tei,1,4583,ATAVILLES,Géographie,Diderot,"* ATAVILLES, s. m. pl. (Géog.) peuples du Péro..."
6,volume01-4987.tei,1,4987,AVIQUIRINA,Géographie,Diderot,"* AVIQUIRINA, (Géog.) île de l'Amérique septen..."
7,volume01-5207.tei,1,5207,AYUTLAN,Géographie,Diderot,"* AYUTLAN, (Géog.) riviere de l'Amérique septe..."
8,volume01-5235.tei,1,5235,AZIRUTH,Géographie,Diderot,"* AZIRUTH (Géographie.) petite ville d'Egypte,..."
9,volume01-5237.tei,1,5237,AZMER,Géographie,Diderot,"* AZMER (Géographie.) ville des Indes, dans le..."


In [5]:
print('There are ' + str(df_geo.shape[0]) + ' geography articles')

There are 32 geography articles


Then, we can also make a query based on the value of the data. For instance, we can query all the articles of a specific author:

In [7]:
val = 'Jaucourt'
n = df_geo.loc[dataset['author'] == val].shape[0]
print(str(n) + ' were written by '+ val)

10 were written by Jaucourt


We can also easily show the number of articles per author

In [8]:
df_geo.groupby(['author'])["filename"].count()

author
Diderot                   12
Jaucourt                  10
d'Alembert & d'Holbach     1
unsigned                   9
Name: filename, dtype: int64

It is possible to show the value of one of the column of our dataframe for a specific row (i.e., article) based on its name. For instance, if we want to know who wrote the article about Lyon or if we want to see its content:

In [9]:
dataset.loc[dataset['head'] == 'LYON'].author.item()

'Jaucourt'

In [10]:
dataset.loc[dataset['head'] == 'LYON'].text.item()

'LYON, (Géogr.) grande, riche, belle, ancienne & celebre ville de France, la plus considérable du royaume après Paris, & la capitale du Lyonnois. Elle se nomme en latin Lugdunum, Lugudunum, Lugdumum Segusianorum, Lugdumum Celtarum, &c. Voyez Lugdunum. Lyon fut fondée l\'an de Rome 712, quarante-un ans avant l\'ere chrétienne, par Lucius Munatius Plancus, qui étoit consul avec AEmilius Lepidus. Il la bâtit sur la Sône, au lieu où cette riviere se jette dans le Rhône, & il la peupla des citoyens romains  qui avoient été chassés de Vienne par les Allobroges. On lit dans Gruter une inscription où il est parlé de l\'établissement de cette colonie ; cependant on n\'honora pas Lyon d\'un nom romain ; elle eut le nom gaulois Lugdun, qu\'avoit la montagne aujourd\'hui Forvieres, sur laquelle cette ville fut fondée. Vibius Sequester prétend que ce mot Lugdun signifioit en langue gauloise, montagne du corbeau. Quoi qu\'il en soit, la ville de Lyon est presque aussi souvent nommée  Lugudunum dans 

We can also perform a keyword search over the text content of all articles:

In [11]:
val = 'france'
df_2 = dataset[dataset['text'].str.contains(val, case=False)]
print(str(df_2.shape[0]) + ' articles contain the word \''+ val + '\'')

11 articles contain the word 'france'


Another example with the expression "ville de" will extract all articles that contain the expression 'ville de':

In [12]:
dataset[dataset['text'].str.contains("ville de", case=False)]

,filename,volume,number,head,normClass,author,text
9,volume01-5237.tei,1,5237,AZMER,Géographie,Diderot,"* AZMER (Géographie.) ville des Indes, dans le..."
11,volume02-1783.tei,2,1783,BESANÇON,Géographie,Diderot,"* BESANÇON, (Géog.) ville de France, capitale ..."
15,volume04-4137.tei,4,4137,DENAT,Géographie moderne,unsigned,"DENAT, (Géog. mod.) petite ville de France au ..."
21,volume09-3630.tei,9,3630,LYON,Géographie,Jaucourt,"LYON, (Géogr.) grande, riche, belle, ancienne ..."
31,volume11-760.tei,11,760,NICOYA,Géographie,unsigned,"NICOYA, (Géogr.) ville de l'Amérique septentri..."
32,volume17-2675.tei,17,2675,"Zacatula, la",Géographie moderne,unsigned,"Zacatula, la, (Géog. mod.) riviere de l'Amériq..."


The same with the words 'océan pacifique' and 'mer pacifique'. Which can be used to study the extent of the Encyclopedie on the pacific area:

In [13]:
dataset[dataset['text'].str.contains("océan pacifique|mer pacifique", case=False)]


,filename,volume,number,head,normClass,author,text
3,volume01-2568.tei,1,2568,ANCUD,Géographie moderne,Diderot,"* ANCUD, (Géog. mod.) l'Archipel d'Ancud ou de..."
4,volume01-4093.tei,1,4093,ARRACIFES,Géographie,Diderot,"* ARRACIFES, (Géog.) une des îles des Larrons,..."
5,volume01-4583.tei,1,4583,ATAVILLES,Géographie,Diderot,"* ATAVILLES, s. m. pl. (Géog.) peuples du Péro..."
6,volume01-4987.tei,1,4987,AVIQUIRINA,Géographie,Diderot,"* AVIQUIRINA, (Géog.) île de l'Amérique septen..."
7,volume01-5207.tei,1,5207,AYUTLAN,Géographie,Diderot,"* AYUTLAN, (Géog.) riviere de l'Amérique septe..."
12,volume03-2596.tei,3,2596,COCOS (Isle des),Géographie moderne,unsigned,"COCOS (Isle des), Géog. mod. île de l'Amérique..."
22,volume09-3890.tei,9,3890,MAGELLANIQUE la terre,Géographie,Jaucourt,"MAGELLANIQUE la terre, (Géog.) C'est ainsi que..."
24,volume10-1660.tei,10,1660,MER,Géographie,d'Alembert & d'Holbach,"MER, s. f. (Géog.) ce terme signifie ordinaire..."
26,volume10-1695.tei,10,1695,Mer du Sud,Géographie,unsigned,"Mer du Sud, (Géog.) vaste partie de l'Océan, e..."
27,volume10-557.tei,10,557,"MARIANES, (les îles)",Géographie,unsigned,"MARIANES, (les îles) autrement les îles DAS VE..."


Then, the same with a more thematic search for instance about 'esclavage': 

In [14]:
dataset[dataset['text'].str.contains("esclavage", case=False)]

,filename,volume,number,head,normClass,author,text
10,volume02-1650.tei,2,1650,Benin,Géographie,Diderot,"* Benin, (Géog.) capitale du royaume de même n..."
13,volume03-3978.tei,3,3978,CONQUÊTE,Droit des gens,Jaucourt,"CONQUÊTE, s. f. (Droit des gens.) acquisition ..."
16,volume05-3381.tei,5,3381,ESCLAVAGE,Droit naturel | Morale | Religion,Jaucourt,"ESCLAVAGE, s. m. (Droit nat. Religion, Morale...."
17,volume05-3382.tei,5,3382,Esclavage,Commerce,Mallet,"Esclavage, (Comm.) On appelle ainsi en Anglete..."
19,volume08-240.tei,8,240,HAMBOURG,Géographie,Jaucourt,"HAMBOURG, (Géog.) Hamburgum, grande & très-ric..."
29,volume11-3175.tei,11,3175,OSTIE,Géographie,Jaucourt,"OSTIE, (Géogr.) ancienne ville d'Italie dans l..."
34,volume17-681.tei,17,681,VICENCE,Géographie moderne,Jaucourt,"VICENCE, (Géog. mod.) en italien Vicenza, en l..."


## 2. The NLP pipeline: Perdido Geoparser

### 2.1 Preprocessing: tokenization and part-of-speech (POS) tagging 

In Natural Language Processing (NLP), the main first steps before processing text content consist in tokenizing sentences and words and assigning to each word its grammatical category (Part-of-Speech). Then, this allows the construction of more complex rules or queries than a simple keyword search.
This preprocessing step is language dependent and thus we have to choose the right tool according to the language of our documents. This is a major difficulty when dealing with historical or ancient texts. For instance, for French it is difficult to find a POS tagger for old French as all well known taggers are trained on contemporary corpora.

> McDonough, K., Moncla, L., & van de Camp, M. (2019). Named entity recognition goes to old regime France: geographic text analysis for early modern French corpora. International Journal of Geographical Information Science, 33, 2498–2522.


The `Perdido` geoparser uses [Treetagger](https://www.cis.uni-muenchen.de/~schmid/tools/TreeTagger/) for part-of-speech tagging. 


### 2.2 Geoparsing: geotagging + geocoding

Geoparsing is divided into two main tasks: geotagging (NER) and geocoding.

The geotagging service of the `Perdido` API uses a cascade of finite-state transducers defining specific patterns for NER and identification of geographic information (spatial relations, etc.). 
> Mauro Gaio and Ludovic Moncla (2019). “Geoparsing and geocoding places in a dynamic space context.“ In The Semantics of Dynamic Space in French: Descriptive, experimental and formal studies on motion expression, 66, 353.

For our custom version of the `Perdido` Geoparser, the geocoding task uses a simple gazetteer lookup method. We use the French wikiGazetteer (a gazetteer based on Wikipedia and enriched with Geonames data) generated following this work: https://github.com/alan-turing-institute/lwm_GIR19_resolving_places/tree/master/gazetteer_construction
> Mariona Coll Ardanuy, Katherine McDonough, Amrey Krause, Daniel CS Wilson, Kasra Hosseini, and Daniel van Strien. (2019) “Resolving Places, Past and Present: Toponym Resolution in Historical British Newspapers Using Multiple Resources”. In Proceedings of the 13th Workshop on Geographic Information Retrieval (GIR19).

Geographic text analysis research in the digital humanities has focused on projects analyzing modern English-language corpora. 
In this tutorial we propose to highlight the difficulties of extracting and mapping geographical information from historical French texts.
As we'll see in the following, in addition to the problem of language when it comes to historical documents, the early-modern period lacks temporally appropriate gazetteers.

> McDonough, K., Moncla, L., & van de Camp, M. (2019). Named entity recognition goes to old regime France: geographic text analysis for early modern French corpora. International Journal of Geographical Information Science, 33, 2498–2522.




The PERDIDO Geoparsing service (`http://erig.univ-pau.fr/PERDIDO/api/geoparsing/`) takes 6 new parameters:
1. api_key: API key of the user
2. lang: language of the document (currently only available for French)
3. content: textual content to parse
4. mode: indicates if the query uses exact match on the name (mode: *s*) or if it uses also alternate names (mode: *a*). (default : *s*)
5. records_limit: maximum number of records found in gazetteer for each toponym (default: 1)
6. version: indicates the version of the geoparser (Encyclopedie or Standard). Default: Standard (the standard version has been developped for the analysis of hiking descriptions)

The PERDIDO Geoparser returns XML-TEI. The `<name>` element refers to named entities (proper nouns) and the type attribute indicates its class (place, person, etc.). The `<rs>` element refers to extended named entities (e.g. ville d'Egypte). The `<location>` element indicates that geographic coordinates were found during geocoding.  



As we'll see in the next cell, when we apply the PERDIDO Geoparser to the following example: (Volume 1 article 5236, available online from the [ARTFL project](https://artflsrv03.uchicago.edu/philologic4/encyclopedie1117/navigate/1/5236/))

>AZIRUTH (Géographie.) petite ville d'Egypte, sur la côte occidentale de la mer Rouge ; ce n'est presque plus qu'un village.


Three spatial entities are found during geotagging:
1. Aziruth, 
2. petite ville d'Egypte
3. la côte occidentale de la mer Rouge

while only one entity (*Egypte*) is found during geocoding:

```xml
<name type="place" subtype="edda" id="en.2">
   <w lemma="null" type="NPr" xml:id="w9">Egypte</w>
   <location>
      <geo source="wiki">35.4833 24.1333</geo>
   </location>
</name>
```

In [15]:
content = dataset.loc[dataset['head'] == 'AZIRUTH'].text.item()
content

"* AZIRUTH (Géographie.) petite ville d'Egypte, sur la côte occidentale de la mer Rouge ; ce n'est presque plus qu'un village."

#### 2.2.1 

In [16]:
geoparser = Geoparser(version="Encyclopedie")
doc = geoparser(content)

In [17]:
doc.tei

'<TEI><teiheader></teiheader><text><body><s><phr type="loc" subtype="relationHead"><phr type="relationHead"><rs type="ene" id="en.0"><rs type="unknown" subtype="ene" id="en.1" start="0" end="0" startT="0" endT="2"><term type="unknown" start="0" end="0" startT="0" endT="1"><w pos="N" lemma="*" id="w0">*</w></term><rs type="place" subtype="no" id="en.2" start="1" end="0" startT="1" endT="2"><name type="place" subtype="edda" id="en.3" start="1" end="0" startT="1" endT="2"><w pos="NPr" lemma="aziruth" id="w1">AZIRUTH</w></name></rs></rs></rs><term type="articleClass" start="1" end="0" startT="2" endT="6"><w pos="PUN" lemma="(" id="w2">(</w><w pos="N" lemma="g&#233;ographie" id="w3">G&#233;ographie</w><w pos="PUN" lemma="" id="w4">.</w><w pos="PUN" lemma=")" id="w5">)</w></term><rs type="ene" id="en.4"><rs type="place" subtype="ene" id="en.5" start="1" end="0" startT="6" endT="10"><term type="place" start="1" end="0" startT="6" endT="8"><w pos="A" lemma="petit" id="w6">petite</w><w pos="N" 

In [18]:
doc.geojson

{'type': 'FeatureCollection',
 'features': [{'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [29.267547, 26.254049]},
   'properties': {'id': 'en.7',
    'name': 'Egypte',
    'sourceName': 'مصر',
    'type': 'administrative',
    'country': 'مصر',
    'source': 'nominatim'}},
  {'type': 'Feature',
   'geometry': {'type': 'Point', 'coordinates': [38.534297, 20.296545]},
   'properties': {'id': 'en.11',
    'name': 'mer Rouge',
    'sourceName': 'البحر الأحمر',
    'type': 'sea',
    'country': 'البحر الأحمر',
    'source': 'nominatim'}}]}

In [19]:
df = doc.to_dataframe()
df.head()

,name,tag,lat,lng,toponym_candidates
0,AZIRUTH,place,None,None,None
1,Egypte,place,29.267547,26.254049,"[{'name': '', 'lat': '29.267547', 'lng': '26.2..."
2,mer Rouge,place,38.534297,20.296545,"[{'name': '', 'lat': '38.534297', 'lng': '20.2..."


#### 2.2.2 Save the results in files

In [20]:
doc.to_xml('aziruth-perdido.xml')

In [21]:
doc.to_geojson('aziruth-perdido.geojson')

In [22]:
doc.to_csv('aziruth-perdido.csv')

#### 2.2.3 Display named entities



https://spacy.io/usage/visualizers#span

In [23]:
displacy.render(doc.to_spacy_doc(), style="ent", jupyter=True)

In [24]:
displacy.render(doc.to_spacy_doc(), style="span", jupyter=True)

#### 2.2.4 Mapping place names


In [25]:
doc.get_folium_map()

## 3. Comparison to other NER tools
### 3.1 SpaCy

In [ ]:
!pip install -U spaCy
!python -m spacy download fr_core_news_sm

In [ ]:
import spacy

In [ ]:
spacy_parser = spacy.load('fr_core_news_sm')

In [ ]:
doc = spacy_parser(content)
for ent in doc.ents:
    print(ent.text, ent.label_)


In [ ]:
displacy.render(doc, style="ent", jupyter=True) 

### 3.2 Stanza

In [ ]:
!pip install stanza

In [ ]:
import stanza

In [ ]:
stanza_parser = stanza.Pipeline(lang='fr', processors='tokenize,ner')

In [ ]:
doc = stanza_parser(content)
print(*[f'entity: {ent.text}\ttype: {ent.type}' for ent in doc.ents], sep='\n')

## 4. Processing several documents at once


In [26]:
df_sampled = dataset.sample(4)
df_sampled

,filename,volume,number,head,normClass,author,text
8,volume01-5235.tei,1,5235,AZIRUTH,Géographie,Diderot,"* AZIRUTH (Géographie.) petite ville d'Egypte,..."
20,volume09-3608.tei,9,3608,LYCUS,Géographie ancienne,Jaucourt,"LYCUS, (Géog. anc.) ce mot est grec, & veut di..."
23,volume10-1324.tei,10,1324,MÉDITERRANÉE,Géographie,unsigned,"MÉDITERRANÉE, s. f. (Géogr.) signifie cette va..."
28,volume11-247.tei,11,247,NATOLIE ou ANATOLIE,Géographie ancienne,Jaucourt,"NATOLIE ou ANATOLIE, (Géog. anc.) on l'appello..."


In [27]:
df_sampled.text

8     * AZIRUTH (Géographie.) petite ville d'Egypte,...
20    LYCUS, (Géog. anc.) ce mot est grec, & veut di...
23    MÉDITERRANÉE, s. f. (Géogr.) signifie cette va...
28    NATOLIE ou ANATOLIE, (Géog. anc.) on l'appello...
Name: text, dtype: object

`geoparser` can parse a string, a list of string or a pandas.Series of string.
When the argument is a list or a pandas.series, the geoparser returns a PerdidoCollection object, while when it is a string it returns a Perdido object.

In [28]:
docs = geoparser(df_sampled.text)

In [14]:
for doc in docs:
    displacy.render(doc.to_spacy_doc(), style="ent", jupyter=True) 

## 5. Load EDdA geoparsed dataset

In [29]:
d = load_edda_perdido()
dataset = d['data']

In [31]:
for index, doc in enumerate(dataset):
    print(dataset.metadata[index])

{'filename': 'volume01-1063.tei', 'volume': '01', 'number': '1063', 'head': 'AFRIQUE', 'normClass': 'Géographie', 'author': 'Diderot'}
{'filename': 'volume01-1468.tei', 'volume': '01', 'number': '1468', 'head': 'AISNE', 'normClass': 'Géographie', 'author': 'Diderot'}
{'filename': 'volume01-1789.tei', 'volume': '01', 'number': '1789', 'head': 'ALLEMAGNE', 'normClass': 'Géographie', 'author': 'Diderot'}
{'filename': 'volume01-2568.tei', 'volume': '01', 'number': '2568', 'head': 'ANCUD', 'normClass': 'Géographie moderne', 'author': 'Diderot'}
{'filename': 'volume01-4093.tei', 'volume': '01', 'number': '4093', 'head': 'ARRACIFES', 'normClass': 'Géographie', 'author': 'Diderot'}
{'filename': 'volume01-4583.tei', 'volume': '01', 'number': '4583', 'head': 'ATAVILLES', 'normClass': 'Géographie', 'author': 'Diderot'}
{'filename': 'volume01-4987.tei', 'volume': '01', 'number': '4987', 'head': 'AVIQUIRINA', 'normClass': 'Géographie', 'author': 'Diderot'}
{'filename': 'volume01-5207.tei', 'volume'

In [34]:
dataset.to_dataframe()

,filename,volume,number,head,normClass,author
0,volume01-1063.tei,01,1063,AFRIQUE,Géographie,Diderot
1,volume01-1468.tei,01,1468,AISNE,Géographie,Diderot
2,volume01-1789.tei,01,1789,ALLEMAGNE,Géographie,Diderot
3,volume01-2568.tei,01,2568,ANCUD,Géographie moderne,Diderot
4,volume01-4093.tei,01,4093,ARRACIFES,Géographie,Diderot
5,volume01-4583.tei,01,4583,ATAVILLES,Géographie,Diderot
6,volume01-4987.tei,01,4987,AVIQUIRINA,Géographie,Diderot
7,volume01-5207.tei,01,5207,AYUTLAN,Géographie,Diderot
8,volume01-5235.tei,01,5235,AZIRUTH,Géographie,Diderot
9,volume01-5237.tei,01,5237,AZMER,Géographie,Diderot


## 3. Toponym disambiguation using network analysis

In our work, we use this methodoly for constructing a network based on the citation of "géographie" articles between them.
We proposed to use network analysis measures to establish an approximate location, defined by qualitative relations, for each named toponym in EDDA. Throwing a list of decontextualized toponyms at an external resource like Geonames is risky. We therefore hypothesize that defining meaningful links between places can provide essentialinformation to improve disambiguation (and potentially replace resolution as the end goal). We establish connections between places based on the citation of “headword” toponyms (those that appearas headwords of entries) in other EDDA entries.

>Moncla, L., McDonough, K., Vigier, D., Joliveau, T., & Brenon, A. (2019). Toponym disambiguation in historical documents using network analysis of qualitative relationships. Proceedings of the 3rd ACM SIGSPATIAL International Workshop on Geospatial Humanities, 1–4. Chicago, IL, USA.

This method draws on relations in the corpus of EDDA articles, which improves disambiguation at a later stage with an external resource. We suggest the network as an alternative to geospatial representation, a useful proxy when no historical gazetteer exists for the source material's period. Our first experiments have shown that this approach goes beyond a simple text analysis and is able to find relations between toponyms that are not co-occurring in the same documents. Network relations are also usefully compared with disambiguated toponyms to evaluate geographical coverage, and the ways that geographical discourse is expressed, in historical texts.


<table>
  <tr>
    <td> <img src="img/labels_indegree2.png" width ="500px"> </td>
    <td> <img src="img/nodes_betweenness+class2.png" width ="500px" > </td>
  </tr>
  <tr>
    <td>Node and label size indicate in-degree centrality</td>
    <td>Node size indicates betweenness centrality<br/> 
        colors refer to geographic feature types <br/> 
        (city: red, hydronym: blue, country: green, mountain: brown, unclassified: grey)</td>
  </tr>
</table> 



We also made somse preliminary tests by assigning geographic coordinates found in our French wikiGazetteer to each node (headword). We have only 2535 nodes with coordinates over the 13734 nodes. 

Our first experiment is shown below. Colors identify clusters of nodes computed with the [modularity measure](https://en.wikipedia.org/wiki/Modularity_(networks)) implemented on Gephy.

<table><tr>
<td> <img src="img/geocodingEDDA1.png" width ="500"> </td>
<td> <img src="img/geocodingEDDA_network.png" width ="500" > </td>
</tr></table> 